## Google Drive

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
print(os.getcwd(), os.listdir(os.getcwd()))

Mounted at /content/drive
/content ['.config', 'drive', 'sample_data']


In [2]:
# you might need to create a google drive SHORTCUT that has this same path
# ... or update the path to use your own google drive organization
DIRPATH = '/content/drive/MyDrive/Research/DS Research Shared 2024'

print(DIRPATH)
os.path.isdir(DIRPATH)

/content/drive/MyDrive/Research/DS Research Shared 2024


True

In [3]:
DATA_DIRPATH = os.path.join(DIRPATH, "projects", "Impeachment 2020 Embeddings", "data")
os.path.isdir(DATA_DIRPATH)

True

## Data Loading

In [4]:
from pandas import read_parquet

pq_filepath = os.path.join(DATA_DIRPATH, "botometer_sample_max_50_openai_status_embeddings_v3_unpacked.parquet.gzip")
statuses_df = read_parquet(pq_filepath)
print(statuses_df.shape)
print(statuses_df.columns)
statuses_df.head()

(183815, 1541)
Index(['user_id', 'status_id', 'status_text', 'created_at', 'embeds_length',
       'openai_0', 'openai_1', 'openai_2', 'openai_3', 'openai_4',
       ...
       'openai_1526', 'openai_1527', 'openai_1528', 'openai_1529',
       'openai_1530', 'openai_1531', 'openai_1532', 'openai_1533',
       'openai_1534', 'openai_1535'],
      dtype='object', length=1541)


,user_id,status_id,status_text,created_at,embeds_length,openai_0,openai_1,openai_2,openai_3,openai_4,...,openai_1526,openai_1527,openai_1528,openai_1529,openai_1530,openai_1531,openai_1532,openai_1533,openai_1534,openai_1535
0,897845802701377536,1221540755451392001,Doubt it..It appears they all have gone the wa...,2020-01-26 21:09:45+00:00,1536,-0.020428,-0.006720,0.007308,-0.022157,-0.041841,...,0.014616,0.004705,0.012661,-0.020974,-0.003458,0.045166,0.029871,-0.021186,-0.003376,-0.024937
1,935739601301458947,1223458629837295619,RT @Wyn1745: Democrats are ‘setting the stage’...,2020-02-01 04:10:42+00:00,1536,-0.036689,-0.007481,0.007968,-0.006632,-0.022805,...,-0.001696,0.002522,0.020397,-0.046374,-0.046611,0.021068,-0.000085,-0.003701,-0.015370,-0.019213
2,571774622,1217445781663363072,RT @sarahdwire: I’m loathe to insert myself in...,2020-01-15 13:57:48+00:00,1536,-0.033382,-0.006886,-0.003244,-0.015834,0.000172,...,0.001027,0.002464,0.002013,-0.032766,-0.034265,0.006545,0.014804,0.003027,-0.001518,-0.030946
3,384679808,1223705594818748416,RT @RepRatcliffe: We warned them...As Schiff a...,2020-02-01 20:32:03+00:00,1536,-0.008477,-0.007364,0.000919,-0.006435,0.008101,...,-0.028269,0.003193,0.015056,-0.015333,-0.028137,0.032510,0.010327,-0.013621,-0.007686,-0.016216
4,701264221653217281,1218459840277729281,"RT @chipfranklin: Because ""impeachment"" in the...",2020-01-18 09:07:18+00:00,1536,-0.009454,0.017376,0.007016,-0.020075,-0.023674,...,-0.013590,0.015564,0.005130,0.003077,-0.029167,0.015523,0.017914,-0.008789,-0.019767,-0.042353


In [17]:
embeddings_cols = [col for col in statuses_df.columns if "openai" in col]
print(len(embeddings_cols))
print(embeddings_cols[0], "...", embeddings_cols[-1])

1536
openai_0 ... openai_1535


In [5]:
statuses_df["user_id"].nunique()

7566

In [6]:
len(statuses_df)

183815

In [7]:
statuses_df["status_id"].nunique()

183727

Oh no, not unique?

In [8]:
statuses_df["status_id"].value_counts()

1234905353650761728    6
1209143341901737984    3
1209173027772076033    3
1207894148151308289    2
1217603880453718016    2
                      ..
1216442996260003840    1
1225979782745272325    1
1206336484170702849    1
1239320120071200771    1
1222940911023333376    1
Name: status_id, Length: 183727, dtype: Int64

## De-Duping

183,727 statuses

In [12]:
print(statuses_df.shape)
statuses_df.drop_duplicates(subset=["status_id"], inplace=True)
print(statuses_df.shape)

(183815, 1541)
(183727, 1541)


## Averaging Embeddings per User

In [14]:
statuses_df.groupby("user_id")["status_id"].count()

user_id
2952                    6
635553                 12
656993                  1
761154                  4
777554                  1
                       ..
1234200349600288772    50
1234846911028453376     1
1237940420136456192     4
1238854780191195136     1
1240138605726760962     1
Name: status_id, Length: 7566, dtype: int64

In [18]:
averages = statuses_df.groupby("user_id")[embeddings_cols].mean()
print(averages.shape)
averages.head()

(7566, 1536)


,openai_0,openai_1,openai_2,openai_3,openai_4,openai_5,openai_6,openai_7,openai_8,openai_9,...,openai_1526,openai_1527,openai_1528,openai_1529,openai_1530,openai_1531,openai_1532,openai_1533,openai_1534,openai_1535
user_id,,,,,,,,,,,,,,,,,,,,,
2952,-0.023816,0.002004,0.004429,-0.019361,-0.009860,0.004878,0.000960,-0.015426,-0.006430,0.001027,...,-0.012285,0.001094,0.015767,-0.026536,-0.024981,0.015113,0.018588,-0.002324,-0.003782,-0.028532
635553,-0.030022,-0.006063,0.017259,-0.018501,-0.008536,0.004416,-0.011840,-0.010581,-0.010859,-0.003771,...,-0.005960,-0.007866,0.010948,-0.021376,-0.023424,0.020705,0.005084,-0.011961,-0.003258,-0.026262
656993,-0.010723,0.008235,0.004192,-0.040441,-0.015172,0.012798,-0.015786,0.008556,-0.022145,-0.017026,...,-0.018110,0.007116,-0.004877,-0.032427,-0.023885,-0.000715,0.003886,-0.024242,0.003839,-0.048883
761154,-0.021389,-0.004747,0.006925,-0.017395,-0.011900,0.018309,-0.007047,-0.024175,0.001368,0.002065,...,0.013326,-0.020819,0.007364,-0.016794,-0.049548,0.013037,0.024798,-0.008543,0.006142,-0.035867
777554,-0.009369,-0.009612,0.012470,0.005079,-0.019303,-0.010459,0.019815,-0.019171,-0.017594,-0.006209,...,0.007358,0.010696,0.008784,-0.024808,-0.008042,0.011077,0.001996,-0.001104,-0.019460,-0.030301


In [19]:
csv_filepath = os.path.join(DATA_DIRPATH, "botometer_sample_max_50_openai_status_embeddings_v3_unpacked_deduped_averaged.csv")

averages.to_csv(csv_filepath)